In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from urllib.parse import quote_plus as urlquote
from sqlalchemy.engine import URL
from IPython.display import display
import os
from datetime import datetime

In [ ]:
def process_files_in_local_folder(campaign_path, result_path):
    """Process files in the campaign folder based on date range."""
    begin = input('Starting date (YYYY-MM-DD or 0 for the beginning): ').strip()
    end = input('Ending date (YYYY-MM-DD or 1 for all): ').strip()

    if begin == '0':
        begin_date = datetime.min
    else:
        begin_date = datetime.strptime(begin, '%Y-%m-%d')

    if end == '1':
        stop_date = datetime.max
    else:
        stop_date = datetime.strptime(end, '%Y-%m-%d')

    campaign_folders = sorted([f for f in os.listdir(campaign_path) if os.path.isdir(os.path.join(campaign_path, f))])

    final_results = pd.DataFrame()

    for campaign in campaign_folders:
        print(f"this is campaing date in campaing folder: {campaign}")
        try:
            campaign_date = datetime.strptime(campaign, '%Y-%m-%d')
        except ValueError:
            print(f"Skipping invalid folder: {campaign}")
            continue

        if not (begin_date <= campaign_date <= stop_date):
            print('went in here')
            continue

        print(f"Processing campaign folder: {campaign}")

        campaign_folder = os.path.join(campaign_path, campaign)
        segments = [f for f in os.listdir(campaign_folder) if os.path.isdir(os.path.join(campaign_folder, f))]

        for segment in segments:
            print(f"Processing segment: {segment}")
        
            segment_path = os.path.join(campaign_folder, segment)
        
            # Define paths for CG files
            cg_path = os.path.join(segment_path, "cg_0.xlsx")
        
            # Check if CG file exists
            if not os.path.exists(cg_path):
                print(f"CG Excel file not found in segment: {segment}")
                continue
        
            # Collect all TG files
            tg_files = [os.path.join(segment_path, f) for f in os.listdir(segment_path) if f.startswith("tg") and f.endswith(".xlsx")]
        
            # Check if TG files exist
            if not tg_files:
                print(f"No TG Excel files found in segment: {segment}")
                continue
        
            try:
                # Read CG file
                df_cg = pd.read_excel(cg_path)
        
                # Read and concatenate all TG files
                df_tg = pd.concat([pd.read_excel(file).iloc[:, :1] for file in tg_files], ignore_index=True)
                df_tg.rename(columns={df_tg.columns[0]: 'phone'}, inplace=True)
            except Exception as e:
                print(f"Error reading Excel files in segment {segment}: {e}")
                continue
        
            # Add distinguishing column
            df_tg['cg_tg'] = 'tg'
            df_cg['cg_tg'] = 'cg'
        
            # Merge CG and TG DataFrames
            merged_df = pd.concat([df_tg, df_cg], ignore_index=True)
            merged_df.drop_duplicates(subset=['phone'], inplace=True)
            print(f"Successfully processed segment: {segment}")

            merged_df['campaign_date'] = campaign_date.strftime('%Y-%m-%d')
            merged_df['segment'] = segment
            display(merged_df)
            start_date, end_date , voucher_value , voucher_usage_count , min_basket , vouchers =  getting_voucher_info_sheet(connection_url , segment , campaign)
            final_merge , total_df = calculate_orders_per_folder(merged_df,connection_url,vouchers ,start_date, end_date )
            metrics_results = calculate_metrics_per_voucher_per_folder(final_merge)
            value = vouchers[0]
            cost = calling_google_sheet(value, campaign, segment)
            final_results_pervoucher = analyzing_metrics_per_voucher(
            metrics_results, pd.DataFrame() ,campaign ,segment, start_date, end_date,
            voucher_value, voucher_usage_count, min_basket, cost , total_df
            )
            final_results = pd.concat([final_results, final_results_pervoucher], ignore_index=True)
    output_file = os.path.join(result_path, 'final_combined_results.xlsx')
    final_results.set_index('start_date', inplace=True)
    final_results.to_excel(output_file, index=True)
    print(f"Processing completed. Results saved to: {output_file}")
    return final_results

def getting_voucher_info_sheet(connection_url , segment , campaign_date):
    df_sheet = zooket_reporting().copy()
    filtered_df = df_sheet[
                (df_sheet['Date'] == campaign_date) & (df_sheet['Segment'] == segment)
            ]

    if not filtered_df.empty:
        voucher_ids = []
        for ids in filtered_df['Voucher ID']:
            voucher_ids.extend(str(ids).split('\n'))
            vouchers = ['100' + str(voucher_id.strip()) for voucher_id in voucher_ids]
        # Convert to integers
        vouchers = [int(voucher) for voucher in vouchers]
    else:
        vouchers = []
    
    print(f"this is from getting_voucher_info_sheet def:{vouchers}")
    with create_engine(connection_url).connect() as connection:
        print('Database Connected')
        date_query = f"""
        SELECT v.start_date as start_date , v.stop_date as end_date , v.customer_quantity , v.value , v.minimum_order_value
        from voucher_table  v
        where v.ids IN ({','.join(map(str, vouchers))});
    """
        total_df_date = pd.read_sql_query(date_query, connection)
    start_date = pd.to_datetime(total_df_date['start_date']).dt.strftime('%Y%m%d')[0]
    end_date = pd.to_datetime(total_df_date['end_date']).dt.strftime('%Y%m%d')[0]
    voucher_value = total_df_date['value']
    voucher_usage_count = total_df_date['customer_quantity']
    min_basket = total_df_date['minimum_order_value']
    return start_date, end_date , voucher_value , voucher_usage_count , min_basket , vouchers
 

def is_convertible_to_int(s):
    try:
        int(s)  
        return True
    except ValueError:
        return False
        
def calculate_orders_per_folder(merged_test,connection_url,vouchers ,start_date, end_date):
    with create_engine(connection_url).connect() as connection:
        print('Database Connected')
        sql_query = f"""
        WITH PreAggregatedData AS (
        SELECT
            phone,
            COUNT(DISTINCT CASE WHEN o.voucher_id IN ({','.join(map(str, vouchers))}) THEN o.user_id END) AS customer_id_actual,
            COUNT(CASE WHEN o.voucher_value = 0 THEN o.id END) AS Org_Order,
            COUNT(CASE WHEN o.voucher_id IN ({','.join(map(str, vouchers))}) THEN o.id END) AS inOrg_Order,
            COUNT(o.id) AS Total_Orders,
            ROUND(SUM(CASE WHEN o.voucher_id IN ({','.join(map(str, vouchers))}) THEN CAST(o.voucher_value AS BIGINT) ELSE 0 END) * 1.0, 2) AS Total_Voucher_Value,
            COUNT(CASE WHEN o.voucher_id IN ({','.join(map(str, vouchers))}) THEN o.voucher_id END) AS Voucher_usage,
            SUM(CAST(o.total_value AS BIGINT)) AS NMV,
            SUM(CAST(o.total_value AS BIGINT)) / NULLIF(COUNT(o.id), 0) AS AOV,
            DATEDIFF(DAY, {start_date}, {end_date}) AS voucher_usage_days
        FROM order_table o with (nolock)
        WHERE 
        o.date BETWEEN {start_date} AND {end_date}
        GROUP BY phone
        ),
        VoucherUsageCounts AS (
        SELECT
            phone,
            COUNT(CASE WHEN Voucher_usage = 1 THEN 1 END) AS Voucher_usage_1,
            COUNT(CASE WHEN Voucher_usage = 2 THEN 1 END) AS Voucher_usage_2,
            COUNT(CASE WHEN Voucher_usage = 3 THEN 1 END) AS Voucher_usage_3,
            COUNT(CASE WHEN Voucher_usage > 3 THEN 1 END) AS Voucher_usage_high
        FROM PreAggregatedData
        GROUP BY phone
        ),
        OrderTimeDetail AS (
        SELECT
            phone,
            o.id AS order_id,
            o.date AS order_date,
            ROW_NUMBER() OVER (PARTITION BY o.phone ORDER BY o.date ASC) AS usage_order
        FROM order_table o with (nolock)
        WHERE o.voucher_id IN ({','.join(map(str, vouchers))})
        AND o.date BETWEEN {start_date} AND {end_date}
        ),
        UsageVoucherTime AS (
        SELECT
            phone,
            MAX(CASE WHEN usage_order = 1 THEN order_date END) AS first_usage_date,
            MAX(CASE WHEN usage_order = 2 THEN order_date END) AS second_usage_date,
            MAX(CASE WHEN usage_order = 3 THEN order_date END) AS third_usage_date
        FROM OrderTimeDetail
        GROUP BY phone
        ),
        AverageUsageDays AS (
            SELECT
                phone,
                sum(CAST(DATEDIFF(DAY, first_usage_date, second_usage_date) AS FLOAT)) AS sum_days_first_second,
                sum(CAST(DATEDIFF(DAY, second_usage_date, third_usage_date) AS FLOAT)) AS sum_days_second_third,
                sum(CAST(DATEDIFF(DAY, first_usage_date, third_usage_date) AS FLOAT)) AS sum_days_first_third
            FROM UsageVoucherTime
            GROUP BY phone
        )
        SELECT
            p.phone,
            p.customer_id_actual,
            p.Org_Order,
            p.inOrg_Order,
            p.Total_Orders,
            p.Total_Voucher_Value,
            p.voucher_usage_days,
            p.Voucher_usage,
            v.Voucher_usage_1,
            v.Voucher_usage_2,
            v.Voucher_usage_3,
            v.Voucher_usage_high,
            p.AOV,
            p.NMV,
            a.sum_days_first_second AS sum_days_first_second,
            a.sum_days_second_third AS sum_days_second_third,
            a.sum_days_first_third AS sum_days_first_third
            FROM PreAggregatedData p
            LEFT JOIN VoucherUsageCounts v ON p.phone = v.phone
            LEFT JOIN AverageUsageDays a ON p.phone = a.phone;
    """
        
        total_df = pd.read_sql_query(sql_query, connection) 
    display(total_df)

    merged_df = merged_test.merge(total_df, on='phone', how='left')
    merged_df.fillna(0, inplace=True)
    return merged_df , total_df
    
def calculate_metrics_per_voucher_per_folder(merged_df):       
    Test_result = pd.pivot_table(
        merged_df, 
        index='cg_tg', 
        values=['Total_Orders', 'phone', 'Total_Voucher_Value', 'inOrg_Order', 'Org_Order','NMV' , 'customer_id_actual','Voucher_usage',
               'Voucher_usage_1', 'Voucher_usage_2', 'Voucher_usage_3' , 'Voucher_usage_high'],  
        aggfunc={
            'Total_Orders': 'sum',  
            'phone': 'count',  
            'Total_Voucher_Value': 'sum',
            'inOrg_Order': 'sum',
            'Org_Order': 'sum',
            'NMV' : 'sum',
            'customer_id_actual':'sum',
            'Voucher_usage': 'sum',
            'Voucher_usage_1': 'sum',
            'Voucher_usage_2': 'sum',
            'Voucher_usage_3': 'sum',
             'Voucher_usage_high' : 'sum'} , 
        fill_value=0  
    )
    Test_result.columns = [' '.join(col).strip() if isinstance(col, tuple) else col for col in Test_result.columns]
    Test_result = Test_result.reset_index()

    result = {}
    categories = Test_result["cg_tg"].unique()
    for metric in Test_result.columns:
        if metric != "cg_tg":
            for cat in categories:
                column_name = f"{metric} {cat}"
                result[column_name] = Test_result.loc[Test_result["cg_tg"] == cat, metric].values[0]

    final_df = pd.DataFrame([result])
    display(final_df)
    return final_df


def zooket_reporting():
    client = gspread.authorize(credentials)
    spreadsheet = client.open_by_url('https://docs.google.com/spreadsheets/')
    worksheet = spreadsheet.worksheet("Express")
    data = worksheet.get_all_values()
    df_sheet = pd.DataFrame(data[1:], columns=data[0])
    return df_sheet

def calling_google_sheet(vouchers, campaign_date, segment ):
    df_sheet_crm = zooket_reporting().copy()
    filtered_df = df_sheet_crm[
        (df_sheet_crm['Date'] == campaign_date) & (df_sheet_crm['Segment'].astype(str) == segment)
    ]
    display(filtered_df)
    if not filtered_df.empty:
        cost = int(filtered_df['Used Credit Count'].values[0].replace(',', ''))
    else:
        print(f"No matching data for this segment: {segment}")
        cost = 0
    return cost
def analyzing_metrics_per_voucher(final_df, final_results,campaign_date ,segment, start_date, end_date, voucher_value, voucher_usage_count, min_basket , cost, total_df):
    # final_df['Share Inorg orders'] = final_df['inOrg_Order tg'] / total_df['inOrg_Order'].sum()
    final_df['start_date'] = campaign_date
    final_df['segment'] = segment
    final_df['min_basket'] = min_basket
    final_df['voucher_usage_count'] = voucher_usage_count
    final_df['voucher_value'] = voucher_value
    final_df['sms_cost'] = cost
    final_df['voucher_usage_days'] = total_df['voucher_usage_days']
    final_df['avg_days_first_third'] = total_df['sum_days_first_third'].sum()/total_df['Voucher_usage_3'].sum()
    final_df['avg_days_second_third'] = total_df['sum_days_second_third'].sum()/total_df['Voucher_usage_3'].sum()
    final_df['avg_days_first_second'] = total_df['sum_days_first_second'].sum()/(total_df['Voucher_usage_2'].sum() + total_df['Voucher_usage_3'].sum())
    final_df['CPO_tg'] = ((final_df['Total_Voucher_Value tg'] + final_df['sms_cost']) / final_df['inOrg_Order tg'])
    final_df['CPO_cg'] = (final_df['Total_Voucher_Value cg'] / final_df['inOrg_Order cg'])
    final_df['Conv Inorg with voucher'] = final_df['inOrg_Order tg'] / final_df['phone tg']
    final_df['Conv Voucher'] = total_df['Voucher_usage'].sum() / final_df['phone tg']
    final_df['voucher_share_once_used'] = total_df['Voucher_usage_1'].sum() / total_df['customer_id_actual'].sum()
    final_df['voucher_share_twice_used'] = total_df['Voucher_usage_2'].sum() / total_df['customer_id_actual'].sum()
    final_df['voucher_share_thrice_used'] = total_df['Voucher_usage_3'].sum() / total_df['customer_id_actual'].sum()
    final_df['voucher_share_high_used'] = total_df['Voucher_usage_high'].sum() / total_df['customer_id_actual'].sum()
    final_df['Conv User-tg'] = final_df['customer_id_actual tg'] / final_df['phone tg']
    final_df['Conv User-cg'] = final_df['customer_id_actual cg'] / final_df['phone cg']
    final_df['Uplift- user'] = (final_df['Conv User-tg'] - final_df['Conv User-cg'])/(final_df['Conv User-cg'])
    final_df['Extra Order'] = final_df['phone tg'] * ((final_df['Total_Orders tg']/final_df['phone tg'])-(final_df['Total_Orders cg']/final_df['phone cg']))
    final_df['Extra GMV'] = final_df['phone tg'] * ((final_df['NMV tg']/final_df['phone tg'])-(final_df['NMV cg']/final_df['phone cg']))
    final_df['Incr Order%'] = final_df['Extra Order']/ final_df['Total_Orders tg']
    final_df['Incr GMV%'] = final_df['Extra GMV']/ final_df['NMV tg']
    final_df['CPI'] = (final_df['Total_Voucher_Value tg']+ final_df['sms_cost'])/final_df['Extra Order']
    final_df['CIG'] = (final_df['Total_Voucher_Value tg'] + final_df['sms_cost'])/final_df['Extra GMV']
    final_df['Org_Share_A'] = final_df['Org_Order tg']/final_df['Total_Orders tg']
    final_df['Org_Share_B'] = final_df['Org_Order cg']/final_df['Total_Orders cg']
    final_df['Cannibalization%'] = (final_df['Org_Order tg']-(final_df['Org_Order cg']*(final_df['phone tg']/final_df['phone cg'])))/final_df['Total_Orders tg']
    final_df['voucher_locking_risk'] = 1-( final_df['inOrg_Order tg'] / total_df['inOrg_Order'].sum())
    final_df['Voucher_health'] = final_df['inOrg_Order tg'] /total_df['Voucher_usage'].sum()
    final_df['User_health'] = final_df['customer_id_actual tg'] /total_df['customer_id_actual'].sum()
    final_df['inorg_opc_tg'] = final_df['inOrg_Order tg']/ final_df['phone tg']
    final_df['inorg_opc_cg'] = final_df['inOrg_Order cg']/ final_df['phone cg']
    final_df['total_opc_tg'] = final_df['Total_Orders tg']/ final_df['phone tg']
    final_df['total_opc_cg'] = final_df['Total_Orders cg']/ final_df['phone cg']
    final_df['total_cpo_tg'] = ((final_df['Total_Voucher_Value tg'] + final_df['sms_cost']) / final_df['Total_Orders tg'])
    final_df['total_cpo_cg'] = (final_df['Total_Voucher_Value cg'] / final_df['Total_Orders cg'])
    column_order = ['start_date','segment','min_basket','voucher_value','voucher_usage_count','voucher_usage_days','phone cg','phone tg','CPI','CIG','Org_Share_A','Org_Share_B','Cannibalization%',
    'voucher_locking_risk','Voucher_health','User_health','avg_days_first_third','avg_days_second_third','avg_days_first_second','CPO_tg','CPO_cg','Conv Inorg with voucher','Conv Voucher',
    'voucher_share_once_used','voucher_share_twice_used','voucher_share_thrice_used','voucher_share_high_used','Conv User-tg','Conv User-cg','Uplift- user','Extra Order','Extra GMV','Incr Order%',
    'Incr GMV%','inorg_opc_tg','inorg_opc_cg','total_opc_tg','total_opc_cg','total_cpo_tg','total_cpo_cg','NMV cg','NMV tg','Org_Order cg','Org_Order tg','Total_Orders cg',
    'Total_Orders tg','Total_Voucher_Value cg','Total_Voucher_Value tg','Voucher_usage cg','Voucher_usage tg','Voucher_usage_1 cg','Voucher_usage_1 tg','Voucher_usage_2 cg','Voucher_usage_2 tg',
    'Voucher_usage_3 cg','Voucher_usage_3 tg','Voucher_usage_high cg',
    'Voucher_usage_high tg','customer_id_actual cg','customer_id_actual tg','inOrg_Order cg','inOrg_Order tg','sms_cost']
    final_df = final_df[column_order]
    final_df.rename(
    columns={
        'phone tg': 'size tg',
        'phone cg': 'size cg',
        'Conv User-tg': 'CR User tg',
        'Conv User-cg': 'CR User cg',
        'Conv Voucher': 'CR Order Total',
        'Conv Inorg with voucher': 'CR Order tg'
    },
    inplace=True
    )

    
    final_results_pervoucher = pd.concat([final_results, final_df], ignore_index=True)

    display(final_results_pervoucher)
    return final_results_pervoucher

def main():
    campaign_test = r"C:\Users\se.ghasemi\Desktop\campaign_test"
    result_path = r"C:\Users\se.ghasemi\Desktop\results"
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    process_files_in_local_folder(campaign_test, result_path)


if __name__ == "__main__":
    main()